In [3]:
import os
print(os.listdir('./'))

['.ipynb_checkpoints', 'aclImdb', 'airlines', 'churn', 'Churn Bank account using XGBoost.ipynb', 'gdp-data-for-indian-states', 'GmapLocation.ipynb', 'imdb-movie-reviews-dataset.zip', 'IndianGDP.ipynb', 'predict Flight Price.ipynb', 'tree.dot', 'tweepy_sample.py', 'Untitled.ipynb', 'Untitled1.ipynb', 'Untitled2.ipynb', 'Untitled3.ipynb', 'Untitled4.ipynb']


## Importing datasets

In [5]:
import pandas as pd

X_ds = pd.read_csv('./churn/Churn_Modelling.csv')

In [6]:
X_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
X_ds.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
x =X_ds.iloc[:,3:13].values
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [9]:
y=X_ds.iloc[:,13].values
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

## Processing categorical data using label encoder

In [12]:

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder_xt = LabelEncoder()

x[:,1] = label_encoder_xt.fit_transform(x[:,1])

In [13]:
x

array([[619, 0, 'Female', ..., 1, 1, 101348.88],
       [608, 2, 'Female', ..., 0, 1, 112542.58],
       [502, 0, 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 0, 'Female', ..., 0, 1, 42085.58],
       [772, 1, 'Male', ..., 1, 0, 92888.52],
       [792, 0, 'Female', ..., 1, 0, 38190.78]], dtype=object)

#### Changing the Gender values 

In [14]:
x[:,2] = label_encoder_xt.fit_transform(x[:,2])

In [15]:
x

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

#### Categorical data for country has 3 values pandas can assume data is in order 0<1<2 so enforcing onehotencoder

In [37]:
import numpy as np
from sklearn.compose import ColumnTransformer

onehotencoder_xt = ColumnTransformer(
    transformers=[
        ("OneHot",        # Just a name
         OneHotEncoder(), # The transformer class
         [1]              # The column(s) to be applied on.
         )
    ],
    remainder='passthrough' # donot apply anything to the remaining columns
)
x = onehotencoder_xt.fit_transform(x)


In [42]:
x = x.astype('int')
x[:,:16]

array([[     1,      0,      0, ...,      1,      1, 101348],
       [     0,      0,      1, ...,      0,      1, 112542],
       [     1,      0,      0, ...,      1,      0, 113931],
       ...,
       [     1,      0,      0, ...,      0,      1,  42085],
       [     0,      1,      0, ...,      1,      0,  92888],
       [     1,      0,      0, ...,      1,      0,  38190]])

## Test train split functions

In [81]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split (x,y,test_size=0.2,random_state=42 )

In [82]:
np.info(y_test)

class:  ndarray
shape:  (2000,)
strides:  (8,)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x1f93ca88e40
byteorder:  little
byteswap:  False
type: int64


In [83]:
import xgboost

classifier = xgboost.XGBClassifier()

classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [84]:
y_pred=classifier.predict(X_test)

In [85]:
y_test

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [86]:
y_pred

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [87]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [88]:
cm

array([[1522,   85],
       [ 196,  197]], dtype=int64)

### Trying to Get Recall, Accuracy and precision
Recall = TP/TP+FN
Accuracy = TP+TN/Total
Precision =TP/TP+FP

[ TP , FP
  FN , TN]

In [89]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
accuracies.mean()
accuracies.std()

0.0105770979006531

In [90]:
accuracies.mean()

0.85375